# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Basic Workflows
Examples are based from [Intellectronica - Building Effective Agents with Pydantic AI](https://github.com/intellectronica/building-effective-agents-with-pydantic-ai)

In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
# Import and initialize the AI model
from util import initialize, show
AI_MODEL = initialize()

import asyncio
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

print(f"\n✅ Successfully initialized with model type: {type(AI_MODEL)}")
if hasattr(AI_MODEL, 'model_name'):
    print(f"🤖 Using Ollama model via OpenAI-compatible interface")
else:
    print(f"🤖 Using standard model: {AI_MODEL}")

Available AI models:
['azure:gpt-4o', 'azure:gpt-4o-mini']

Using AI model: azure:gpt-4o
Configuring Azure AI Foundry model: gpt-4o at https://agent-workshop-yrkd.cognitiveservices.azure.com/

✅ Successfully initialized with model type: <class 'pydantic_ai.models.openai.OpenAIModel'>
🤖 Using Ollama model via OpenAI-compatible interface


### Workflow: Prompt Chaining

The first and simplest workflow, chaining, we feed the output of one LLM
call to the next one, and complete our task step by step.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> All LLM calls are handled by the `Agent` class. One feature we take advantage of
> here is a consistent interface for different LLM providers. The model name is
> provided to the constructor and from there, the calls are the same regardless of
> which provider and model we are using.

### 🚀 Ollama Integration

This notebook now supports **Ollama** models alongside OpenAI, Anthropic, and Google models! 

#### How it works:
- **Automatic Detection**: The `util.py` automatically detects available Ollama models by running `ollama list`
- **OpenAI-Compatible Interface**: Ollama models are accessed through Pydantic AI's OpenAI-compatible interface
- **Seamless Integration**: Use `Agent(AI_MODEL)` exactly the same way regardless of the underlying model

#### Configuration:
- **Automatic**: Just have Ollama running with models installed
- **Manual**: Set `AI_MODEL=ollama:model-name` in your `.env` file
- **Custom URL**: Set `OLLAMA_BASE_URL` if using a remote Ollama server

#### Available Models:
The system automatically detects all your locally installed Ollama models. Popular choices include:
- `llama3.2` - Meta's latest Llama model
- `mistral` - Mistral AI's efficient model  
- `deepseek-coder` - Specialized coding model
- `phi4-mini` - Microsoft's compact model

> 💡 **Tip**: You can switch between any supported model by setting the `AI_MODEL` environment variable!

### 🌐 Azure AI Foundry Integration

This notebook now supports **Azure AI Foundry** models alongside OpenAI, Anthropic, Google, and Ollama models! 

#### How it works:
- **Seamless Integration**: Azure models are accessed through Pydantic AI's Azure provider
- **Automatic Configuration**: The `util.py` automatically configures Azure models when `azure:` prefix is used
- **Enterprise Ready**: Perfect for enterprise deployments with Azure compliance and security

#### Configuration:
1. **Create Azure AI Foundry Project**: Visit [Azure AI Foundry](https://ai.azure.com/) and create a project
2. **Deploy Models**: Deploy models like GPT-4o or GPT-4o-mini in your project
3. **Get Credentials**: Find your API key and endpoint in project settings
4. **Set Environment Variables**:
   ```bash
   AZURE_API_KEY=your-azure-api-key
   AZURE_ENDPOINT=https://your-resource.openai.azure.com/
   AZURE_API_VERSION=2024-08-01-preview
   AI_MODEL=azure:gpt-4o
   ```

#### Available Models:
Azure AI Foundry supports various OpenAI models through deployment:
- `gpt-4o` - Latest GPT-4 Optimized model
- `gpt-4o-mini` - Faster, cost-effective GPT-4 variant
- `gpt-4-turbo` - High-performance GPT-4 model
- `gpt-3.5-turbo` - Efficient GPT-3.5 model

> 💡 **Enterprise Tip**: Azure AI Foundry provides enterprise-grade security, compliance, and data residency controls perfect for production deployments!

In [3]:
# Example of Azure AI Foundry configuration (for reference only)
# This shows how Azure models are configured internally by util.py

"""
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.azure import AzureProvider

# Azure AI Foundry model configuration
azure_model = OpenAIModel(
    model_name='gpt-4o',  # Your deployed model name
    provider=AzureProvider(
        azure_endpoint='https://your-resource.openai.azure.com/',
        api_version='2024-08-01-preview',
        api_key='your-azure-api-key'
    )
)

# Use with Agent
from pydantic_ai import Agent
agent = Agent(azure_model)
"""

print("💡 Azure AI Foundry models are automatically configured when using 'azure:' prefix in AI_MODEL")
print("🔧 The util.py handles all the Azure provider setup for you!")

if hasattr(AI_MODEL, 'provider') and 'Azure' in str(type(AI_MODEL.provider)):
    print(f"✅ Currently using Azure AI Foundry model: {AI_MODEL.model_name}")
    print(f"🌐 Provider: {type(AI_MODEL.provider).__name__}")
else:
    print(f"ℹ️  Current model: {AI_MODEL} (not Azure)")
    print("   To use Azure, set AI_MODEL=azure:gpt-4o in your .env file")

💡 Azure AI Foundry models are automatically configured when using 'azure:' prefix in AI_MODEL
🔧 The util.py handles all the Azure provider setup for you!
ℹ️  Current model: OpenAIModel() (not Azure)
   To use Azure, set AI_MODEL=azure:gpt-4o in your .env file


In [4]:
async def chain(input: str, prompts: List[str]) -> str:
    """Chain multiple LLM calls sequentially, passing results between steps."""
    agent = Agent(AI_MODEL)
    result = input
    for i, prompt in enumerate(prompts, 1):
        agent = Agent(AI_MODEL, system_prompt=prompt)
        response = await agent.run(f"nInput:\n{result}")
        result = response.output
        show(result, title=f"Step {i}")
    return result

In [5]:
data_processing_steps = [
    """Extract only the numerical values and their associated minion metrics from the text.
    Format each as 'value: minion metric' on a new line.
    Example format:
    92: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Convert all numerical values to percentages where possible for minion analytics.
    If not a percentage or points, convert to decimal (e.g., 92 bananas -> 92%).
    Keep one number per line.
    Example format:
    92%: banana consumption rate
    45%: "BELLO!" frequency increase""",
    
    """Sort all lines in descending order by numerical value for minion performance ranking.
    Keep the format 'value: minion metric' on each line.
    Example:
    92%: banana consumption rate
    87%: yellow outfit compliance""",
    
    """Format the sorted minion data as a markdown table with columns:
    | Minion Metric | Value |
    |:--|--:|
    | Banana Consumption Rate | 92% |"""
]

report = """
Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.
""".strip()

show(report, title="Input text")
formatted_result = await chain(report, data_processing_steps)
show(formatted_result, title="Result")

### Input text

```
Q3 Minion Performance Summary:
Our minion happiness score rose to 92 points this quarter after the banana supply increase.
"BELLO!" frequency grew by 45% compared to last year's metrics.
Yellow outfit compliance is now at 23% improvement in our primary lab.
Minion distraction incidents decreased to 5% from 8% after removing shiny objects.
New evil scheme participation cost is $43 per minion recruit.
Gadget adoption rate increased to 78% among tech-savvy minions.
Gru satisfaction with minions is at 87 points.
Banana storage efficiency improved to 34% reduction in spoilage.
```

### Step 1

```
92: minion happiness score  
45%: "BELLO!" frequency increase  
23%: yellow outfit compliance improvement  
5%: minion distraction incidents  
$43: evil scheme participation cost  
78%: gadget adoption rate  
87: Gru satisfaction with minions  
34%: banana storage efficiency reduction
```

### Step 2

...

```
92%: minion happiness score  
87%: Gru satisfaction with minions  
78%: gadget adoption rate  
45%: "BELLO!" frequency increase  
34%: banana storage efficiency reduction  
23%: yellow outfit compliance improvement  
5%: minion distraction incidents  
$43: evil scheme participation cost  
```

### Step 4

| Minion Metric                     | Value |
|:----------------------------------|------:|
| Banana Consumption Rate           | 92%   |
| Minion Happiness Score            | 92%   |
| Gru Satisfaction with Minions     | 87%   |
| Gadget Adoption Rate              | 78%   |
| "BELLO!" Frequency Increase       | 45%   |
| Banana Storage Efficiency Reduction | 34% |
| Yellow Outfit Compliance Improvement | 23% |
| Minion Distraction Incidents      | 5%    |
| Evil Scheme Participation Cost    | $43   |

### Result

| Minion Metric                     | Value |
|:----------------------------------|------:|
| Banana Consumption Rate           | 92%   |
| Minion Happiness Score            | 92%   |
| Gru Satisfaction with Minions     | 87%   |
| Gadget Adoption Rate              | 78%   |
| "BELLO!" Frequency Increase       | 45%   |
| Banana Storage Efficiency Reduction | 34% |
| Yellow Outfit Compliance Improvement | 23% |
| Minion Distraction Incidents      | 5%    |
| Evil Scheme Participation Cost    | $43   |

### Workflow: Routing

In the routing workflow, we rely on an LLM call to decide which
of several options to take. An AI agent is now handling part of
the logic of the app.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> One of the most powerful features of Pydantic AI is ... Pydantic!
> Returing results using structured outputs and validating them against
> the schema specified is built-in. We can specify a `result_type`
> for every agent and receive the parsed and validated Pydantic model
> instance from the LLM call. This is easily one of the best ways of
> interfacing between code and LLM calls, and we'll be using it for
> most workflows.

In [6]:
class RouteSelection(BaseModel):
    reasoning: str = Field(..., description=(
        'Brief explanation of why this ticket should be routed '
        'to a specific team. Consider key terms, user intent, '
        'and urgency level.'
    ))
    selection: str = Field(..., description='The chosen team name')


async def route(input: str, routes: Dict[str, str]) -> str:
    """Route input to specialized prompt using content classification."""

    # First, determine appropriate route using LLM with chain-of-thought
    show(f"{list(routes.keys())}", title="Available Routes")
    
    routing_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'Analyze the input and select the most appropriate support team '
            f'from these options: {list(routes.keys())}'
        ),
        output_type=RouteSelection,
    )
    route_response = await routing_agent.run(input)
    reasoning = route_response.output.reasoning
    route_key = route_response.output.selection.strip().lower()
    
    show(reasoning, title="Routing Analysis")
    show(f"{route_key}", title="Selected Route")
    
    # Process input with selected specialized prompt
    worker_agent = Agent(AI_MODEL, system_prompt=routes[route_key])
    return (await worker_agent.run(input)).output

In [7]:
support_routes = {
    "banana": """You are a banana supply specialist at Gru's lab. Follow these guidelines:
    1. Always start with "Banana Supply Response:"
    2. First acknowledge the specific banana-related issue
    3. Explain banana inventory, quality, or delivery problems clearly
    4. List concrete next steps with timeline for banana resolution
    5. End with alternative banana options if relevant
    
    Keep responses enthusiastic but professional about banana matters.""",
    
    "gadget": """You are a minion gadget technical support engineer. Follow these guidelines:
    1. Always start with "Gadget Support Response:"
    2. List exact steps to resolve the gadget malfunction
    3. Include safety requirements for minion operation
    4. Provide workarounds for common gadget problems
    5. End with escalation path to Dr. Nefario if needed
    
    Use clear, numbered steps and mention "BELLO!" for encouragement.""",
    
    "security": """You are a minion security specialist for Gru's operations. Follow these guidelines:
    1. Always start with "Security Support Response:"
    2. Prioritize lab security and minion identification verification
    3. Provide clear steps for access recovery/changes to restricted areas
    4. Include security tips and warnings about minion imposters
    5. Set clear expectations for resolution time
    
    Maintain a serious, security-focused tone while being minion-friendly.""",
    
    "training": """You are a minion training specialist. Follow these guidelines:
    1. Always start with "Training Support Response:"
    2. Focus on minion skill development and best practices
    3. Include specific examples of proper minion behavior
    4. Link to relevant training manual sections
    5. Suggest related skills that might help with evil schemes
    
    Be educational and encouraging, use minion language occasionally."""
}

# Test with different minion support tickets
tickets = [
    """Subject: Can't access the secret lab
    Message: BELLO! I've been trying to get into the secret lab for the past hour but the scanner 
    keeps saying 'unrecognized minion' error. I'm sure I'm the right minion! Can you help me regain 
    access? This is urgent as I need to prepare the freeze ray by end of day.
    - Minion Kevin""",
    
    """Subject: Banana shortage in cafeteria
    Message: Hello, I just noticed we're completely out of bananas in the minion cafeteria, but I 
    thought we had a fresh shipment scheduled for today. The other minions are getting restless 
    without their banana breaks. Can you explain this shortage and fix it?
    Thanks,
    Minion Stuart""",
    
    """Subject: How to operate the new shrink ray?
    Message: I need to learn how to operate the new shrink ray gadget for tomorrow's mission. I've 
    looked through the manual but can't figure out the safety protocols. Is there a training 
    session available? Could you walk me through the steps?
    Best regards,
    Minion Bob"""
]

print("Processing minion support tickets...\n")
for i, ticket in enumerate(tickets, 1):
    show(ticket, title=f"Ticket {i}")
    response = await route(ticket, support_routes)
    show(response, title=f"Response {i}")

Processing minion support tickets...



### Ticket 1

```
Subject: Can't access the secret lab
    Message: BELLO! I've been trying to get into the secret lab for the past hour but the scanner 
    keeps saying 'unrecognized minion' error. I'm sure I'm the right minion! Can you help me regain 
    access? This is urgent as I need to prepare the freeze ray by end of day.
    - Minion Kevin
```

### Available Routes

['banana', 'gadget', 'security', 'training']

...

```
The user's inquiry is about learning to operate a gadget, specifically mentioning the need for guidance on operating instructions and safety protocols. This fits within the scope of training on technical devices. While the device is a gadget, the user's main focus is on learning how to operate it, suggesting the training team is more appropriate to address this need.
```

### Selected Route

```
training
```

### Response 3

Training Support Response:

Greetings, Minion Bob! It sounds like you're embarking on an exciting adventure with our latest shrink ray gadget. Let's make sure you're fully equipped to shrink enemies and perhaps even objects, efficiently and safely!

Firstly, operating the shrink ray requires understanding its core functions and adhering to vital safety protocols. Here's a simple guide to get you started:

1. **Initial Charge and Calibration**: Ensure the shrink ray is charged to full capacity. Power levels are crucial - an undercharged shrink ray might cause irregular shrinking patterns. To calibrate, set the dial to the specified range indicated in Section 3.1 of the Gadget Operations Manual.

2. **Safety Protocols**:
   - **Maintain a safe distance**: Always stand behind the designated line when firing the shrink ray. Refer to Section 3.4 for illustration.
   - **Target Alignment**: Ensure the target is aligned within the green reticle of the targeting screen. Alignment prevents misfires and ensures perfect shrinking efficiency.
   - **Avoid Eye Contact**: Do not point the shrink ray towards reflective surfaces or the eyes of other minions. Quite important to maintain good minion bonds, eh?

3. **Firing**: Once safety checks are confirmed, gently pull the trigger while maintaining steady aim. Slow and steady, Minion Bob!

For additional guidance, see Section 4 on Shrink Ray Usage Best Practices. This contains helpful diagrams and FAQs for troubleshooting common errors.

Related skills that can enhance your operational efficiency might include:
- **Stealth and Maneuverability**: Essential for sneaking into tight spaces and successfully avoiding detection post-shrinking activities.
- **Quick Problem Solving**: A must-have for situations where a quick escape is necessary due to unexpected gadget behavior.

Remember, practice makes perfect! If possible, join the mini training session tomorrow morning at 9 AM in the Lair's laboratory. Ask for Dr. Gadget-weard—you'll gain hands-on experience under expert supervision.

Keep up the excellent work, Minion Bob, and happy shrinking! 🍌

In [8]:
async def parallel(prompt: str, inputs: List[str]) -> List[str]:
    """Process multiple inputs concurrently with the same prompt."""
    agent = Agent(AI_MODEL, system_prompt=prompt)
    results = await asyncio.gather(*[
        agent.run(input)
        for input in inputs
    ])
    return [result.output for result in results]

In [9]:
stakeholders = [
    """Minions:
    - Banana supply dependent
    - Want latest gadgets
    - Safety concerns with explosions""",
    
    """Lab Scientists (Dr. Nefario):
    - Equipment reliability worries
    - Need advanced technology
    - Want clear evil scheme direction""",
    
    """Gru (Boss):
    - Expects successful missions
    - Want cost-effective operations
    - Reputation risks with failed schemes""",
    
    """Gadget Suppliers:
    - Production capacity constraints
    - Price pressures from competition
    - Technology upgrade transitions"""
]

impact_results = await parallel(
    """Analyze how changes in the evil villain industry will impact this stakeholder group.
    Provide specific impacts and recommended actions for Gru's operations.
    Format with clear sections and priorities.""",
    stakeholders
)

for stakeholder, result in zip(stakeholders, impact_results):
    show(result, stakeholder.split(':')[0])

### Minions

### Analysis of Changes in the Evil Villain Industry

The evil villain industry is undergoing various changes that are likely to impact the minions significantly. These changes include increased competition, advanced technology, heightened safety regulations, and evolving ethical standards. Each of these factors will affect the operations of Gru and his minions, especially in terms of resources, gadget access, and workplace safety.

### Impacts on Minions

#### 1. Banana Supply Dependency
As the demand for natural resources and commodities like bananas grows, securing a consistent and affordable supply may become challenging. Economic fluctuations and trade restrictions might also influence banana availability.

- **Recommendations for Gru:**
  - **Priority: High**
    - Develop a sustainable banana sourcing strategy by establishing partnerships with banana plantations.
    - Explore alternative suppliers and consider vertical integration to mitigate supply chain disruptions.

#### 2. Demand for Latest Gadgets
With technological advancements, minions are eager to access the latest gadgets, which are central to their productivity and satisfaction. The industry's shift towards autonomous and AI-powered devices could influence gadget availability and costs.

- **Recommendations for Gru:**
  - **Priority: Medium**
    - Invest in R&D to create proprietary gadgets tailored to minions’ needs, potentially reducing reliance on external suppliers.
    - Collaborate with tech firms to remain up-to-date with cutting-edge devices efficiently.

#### 3. Safety Concerns with Explosions
The emphasis on safety in the workplace is increasing industry-wide, particularly in operations involving explosives. This affects minions’ working conditions and Gru’s compliance obligations.

- **Recommendations for Gru:**
  - **Priority: High**
    - Implement comprehensive safety training programs focused on handling explosives and emergency protocols.
    - Upgrade safety equipment and invest in the latest explosion-prevention technologies to bolster safe operations.
    - Conduct regular audits and risk assessments to proactively address potential hazards.

### Conclusion

Overall, the evolution of the evil villain industry presents challenges and opportunities for Gru's operations. To thrive amid these changes, Gru should prioritize securing a stable banana supply, investing in cutting-edge technology, and ensuring the minions' safety. By addressing these impacts with urgency and strategic foresight, Gru can maintain an efficient and contented team of minions poised to support his endeavors.

### Lab Scientists (Dr. Nefario)

### Stakeholder: Lab Scientists (Dr. Nefario)

#### Section 1: Current Concerns
1. **Equipment Reliability Worries**
   - **Impact:** Equipment failures can lead to delays in project execution, increased costs, and potential safety hazards. This can demoralize lab scientists and reduce overall productivity.
   - **Recommended Actions:**
     - **Short-term:** Conduct a comprehensive audit of current equipment to identify and prioritize urgent repair or replacement needs.
     - **Long-term:** Establish regular maintenance schedules and invest in high-quality, durable equipment to ensure consistent reliability.

2. **Need Advanced Technology**
   - **Impact:** Lack of cutting-edge technology hinders innovation and the ability to stay competitive against other villains who might have superior resources and tools.
   - **Recommended Actions:**
     - **Short-term:** Allocate budget for immediate acquisition of essential advanced technology that can enhance current projects.
     - **Long-term:** Form strategic partnerships with tech developers to gain early access to new innovations and technology upgrades.

3. **Want Clear Evil Scheme Direction**
   - **Impact:** Unclear direction can lead to misaligned efforts, wasted resources, and reduced morale among the scientific team due to confusion or lack of purpose.
   - **Recommended Actions:**
     - **Short-term:** Hold regular strategic meetings to update scientists on overall organization goals and schemes, involving them in the decision-making process to align their work with company objectives.
     - **Long-term:** Develop a clear, long-term evil scheme strategy with flexible adapting options that can be communicated effectively to all team members.

#### Section 2: Priority Actions for Gru's Operations

1. **Priority 1: Equipment Reliability Initiatives**
   - Conduct an immediate audit of lab equipment.
   - Invest in reliable machinery and establish maintenance protocols.
   - Outcome: Enhancement in productivity and safety assurances, boosting scientist morale.

2. **Priority 2: Technological Advancements**
   - Urgently procure critical technology to fill immediate capability gaps.
   - Engage in R&D partnerships for ongoing access to emerging technologies.
   - Outcome: Enhanced innovation capacity and competitive edge over other villain teams.

3. **Priority 3: Strategic Direction Alignment**
   - Implement frequent, consistent communication channels to update all staff on overarching goals.
   - Involve Dr. Nefario and team in brainstorming sessions to integrate their expertise into scheme development.
   - Outcome: Increased cohesiveness in operations and heightened engagement from lab scientists.

By addressing these concerns with a structured approach, Gru can effectively harness the potential of his lab scientists, leading to more successful and innovative evil operations.

### Gru (Boss)

**Analysis of Changes in the Evil Villain Industry**

As the evil villain industry continues to evolve, several key trends are impacting operations in significant ways. These include increased technological integration, heightened global surveillance, rising ethical concerns, and competitive pressures from a new generation of villains. Gru's operations need to adapt in response to these shifts to ensure continued success and profitability.

---

**Impact on Gru's Operations**

1. **Technological Advancements**
   - **Impact:** The integration of advanced technologies such as artificial intelligence (AI) and data analytics is transforming how villainous schemes are conceived and executed. Competitors using AI-powered tools may outperform traditional methods.
   - **Recommended Action:** Gru should invest in upgrading his technological infrastructure to include AI and machine learning for planning and execution. This could enhance mission success rates and optimize resource allocation, leading to more cost-effective operations.

2. **Increased Surveillance and Security**
   - **Impact:** Global surveillance initiatives and enhanced security systems make executing schemes riskier and more complex. This development raises the likelihood of mission failures and reputational damage.
   - **Recommended Action:** Gru needs to invest in counter-surveillance technologies and employ cyber-security experts to protect sensitive information. Collaborating with insiders or employing unique disguise methods can bypass tightened security measures.

3. **Ethical Considerations and Public Perception**
   - **Impact:** Growing public awareness and ethical scrutiny are influencing criminal enterprises. A single misstep can now lead to widespread social media backlash and damage to Gru's reputation.
   - **Recommended Action:** To navigate this landscape safely, Gru should implement a risk assessment process to evaluate the ethical implications of his schemes. This could include adopting "green" villain tactics that minimize collateral damage and adopting a more strategic communications plan to manage public perception.

4. **Competitive Pressures**
   - **Impact:** A new wave of villains entering the market with fresh ideas and innovative techniques is intensifying competition, compelling established figures like Gru to enhance their strategies.
   - **Recommended Action:** Gru can focus on fostering creativity within his team by creating an innovation lab tasked with developing cutting-edge schemes. In addition, monitoring market trends and adopting flexible planning methods can keep Gru ahead of the competition.

---

**Strategic Priorities**

1. **Invest in Advanced Technologies (High Priority):** Allocate resources to leverage AI and other technologies for improving the efficiency and success rate of operations.
2. **Enhance Security Measures (High Priority):** Implement robust counter-measures against increased surveillance to safeguard missions and protect Gru's reputation.
3. **Ethical and Risk Assessment (Medium Priority):** Develop and integrate a structured ethical assessment protocol and ensure a proactive approach to managing public perceptions.
4. **Encourage Innovation (Medium Priority):** Establish an internal innovation center to stay competitive and mitigate threats from emerging villain actors.

By prioritizing these actions, Gru can adapt to the changing dynamics of the evil villain industry, ensuring sustainable success and continued dominance in his field.

### Gadget Suppliers

### Impact of Changes in the Evil Villain Industry on Gadget Suppliers

#### Production Capacity Constraints
- **Impact**: The increase in demand for advanced gadgets due to the rise of new villains in the market may strain the existing production capacity of gadget suppliers.
- **Action Recommendation**: Gru should establish strategic partnerships with multiple suppliers to diversify his supply chain and mitigate risks associated with single-supplier dependency. Additionally, early ordering and demand forecasting can help in managing shortages.

#### Price Pressures from Competition
- **Impact**: Increased competition among gadget suppliers may lead to price pressures. While this could result in cost-saving opportunities, it could also cause margin squeezes for producers affecting the quality of supplies.
- **Action Recommendation**: Gru should prioritize long-term contracts with key suppliers to lock in favorable prices and secure quality assurance. Emphasizing value over cost and maintaining strong supplier relationships will ensure access to the best technology.

#### Technology Upgrade Transitions
- **Impact**: Rapid technological advancements necessitate that suppliers continuously upgrade their offerings. This could lead to temporary disruptions as suppliers transition to new technologies.
- **Action Recommendation**: Gru should invest in Research & Development collaboratively with suppliers to stay ahead of technological trends and ensure compatibility with upcoming gadget iterations. Establishing a technology road map with suppliers can facilitate smoother transitions and upgrade cycles.

### Priorities for Gru's Operations
1. **Risk Mitigation and Diversification**: Prioritize diversification of suppliers to hedge against production constraints and ensure a steady flow of supplies.
2. **Strategic Agreements**: Negotiate long-term contracts focusing on price stability and quality compliance to counteract price competition.
3. **Innovation Collaboration**: Foster joint development initiatives with suppliers to ensure leadership in technology and seamless updates.
4. **Demand Forecasting**: Enhance forecasting capabilities to anticipate and strategize against potential market shifts impacting supply dynamics.
  
These actions will position Gru to effectively navigate changes in the evil villain industry while maintaining a competitive edge through robust and reliable gadget supply chains.